In [15]:
# Set the seed value for the notebook so the results are reproducible

from numpy.random import seed
seed(1)

In [16]:
# import necessary libraries

import warnings
warnings.simplefilter('ignore')

import numpy as np

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras import optimizers

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification

In [17]:
# load pandas DataFrame

pd.options.display.max_columns = None    #shows all columns in dataframe

allDataDF = pd.read_csv("../combined_data.csv")
allDataDF.head()

,win shares,mins played (per game),player,games,year,rank,points(per game),ws/48,box +/-,value overall replacement player,team,college,years,mins played (total),points (total),class,player_id,college_id,word,name,big_conf_flag,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,48.0,30.6,Kenyon Martin,757,2000,1,12.3,0.100,0.7,15.5,NJN,cincinnati,15,23134,9325,Player,kenyon-martin-1,cincinnati,cincinnati,Kenyon Martin,0,142,292,0.587,0.586,0.494,0.581,116,81,2042.0,1279,124,0.222000,2.0,869,0.592,14.3,139.0,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,0.3,15.6,A.J. Guyton,80,2000,32,5.5,0.010,-5.4,-1.1,CHI,indiana,3,1246,442,Player,aj-guyton-1,indiana,indiana,A.J. Guyton,1,403,52,0.541,0.455,0.249,0.790,128,73,3260.0,2100,129,0.414000,283.0,429,0.572,13.6,218.0,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,11.6,14.3,Jake Voskuhl,450,2000,33,4.0,0.087,-2.2,-0.3,CHI,uconn,9,6443,1814,Player,jake-voskuhl-1,connecticut,uconn,Jake Voskuhl,1,124,193,0.542,0.542,0.555,0.656,138,83,2184.0,863,86,0.329162,0.0,880,0.573,23.0,160.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,0.6,18.7,Khalid El-Amin,50,2000,34,6.3,0.030,-3.8,-0.4,CHI,uconn,1,936,314,Bust,khalid-el-amin-1,connecticut,uconn,Khalid El-Amin,1,479,10,0.486,0.416,0.257,0.822,108,70,2149.0,1650,186,0.355000,195.0,319,0.527,15.6,188.0,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,-0.1,10.6,Mike Smith,17,2000,35,3.0,-0.033,-5.5,-0.2,WAS,university of louisiana at monroe,1,180,51,Bust,mike-smith-1,louisiana monroe,louisiana,Mike Smith,0,147,42,0.507,0.428,0.357,0.769,55,80,1938.0,1062,90,0.318000,130.0,449,0.551,20.0,241.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200


In [18]:
# Split the x variables from the y variable.
# Features with # are INCLUDED (not dropped) indepedent variables

X = allDataDF[[
              "field_goal_percentage",
              "free_throw_attempt_rate",
              "height",
              "three_point_percentage",
              "true_shooting_percentage",
              "win_shares_per_40_minutes"
             ]]
              
y = allDataDF["class"]
print(f"Number of Rows and Independent(X) Variable: {X.shape}, \n Number of Rows (One Dependent(Y) variable): {y.shape}")

Number of Rows and Independent(X) Variable: (853, 6), 
 Number of Rows (One Dependent(Y) variable): (853,)


In [19]:
X.head()

,field_goal_percentage,free_throw_attempt_rate,height,three_point_percentage,true_shooting_percentage,win_shares_per_40_minutes
0,0.586,0.494,81,0.222000,0.592,0.346
1,0.455,0.249,73,0.414000,0.572,0.145
2,0.542,0.555,83,0.329162,0.573,0.237
3,0.416,0.257,70,0.355000,0.527,0.158
4,0.428,0.357,80,0.318000,0.551,0.165


In [20]:
# Label encoding for Dependent(Y) Variable

label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original 

------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
-----

In [21]:
# Scale the Independent(X) Variables

X_scaler = StandardScaler().fit(X)

#X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

X_scaled = X_scaler.transform(X)

encoded_y = encoded_y   #Use this Y array for the KFold cell far below

In [22]:
# Create a normal neural network with 12 inputs (12 independent variables), 6 hidden nodes, and 4 outputs
# input_dim refers to number of independent(X) variables
# units is output shape i.e. number of categories in the Dependent Variable Column
# In this example, final units=4 because Four Rankings: Star, Above Average, Below Average, Bust

model = Sequential()
model.add(Dense(units=20, activation='relu', input_dim=X.shape[1]))
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=len(y.unique()), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_5 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 22        
Total params: 637
Trainable params: 637
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model

sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
             )

In [24]:
# # scikit-learn k-fold cross-validation (making multiple training datasets)

kfold_scores = []
kf = KFold(n_splits=5, random_state=1, shuffle=True)   # Define the split - into 10 folds 
kf.get_n_splits(allDataDF)   # returns the number of splitting iterations in the cross-validator

for train_index, test_index in kf.split(X):
    #print("Train Index:", train_index) 
    #print("Test Index:", test_index)
    X_train, X_test, y_train, y_test = X_scaled[train_index], X_scaled[test_index], encoded_y[train_index], encoded_y[test_index]
    model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=10,
    shuffle=True,
    verbose=0
    )
    kfold_scores.append(model.evaluate(X_test, y_test))

170/170 [==============================] - 0s 27us/sample - loss: 0.8056 - acc: 0.7353


In [25]:
# Save the model

model.save("Draft_Machine_NeuralNetwork.h5")

In [26]:
# Load the Draft_Data into Pandas

draftDF = pd.read_csv("Draft_data.csv")
draftDF.head()

,rank,player_id,name,college,year,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,1,anthony-edwards-2,Anthony Edwards,missouri state,Fr.,68,12,0.485,0.411,0.328,0.746,23,77,730,452,33,0.318,55,127,0.526,12.1,59,0.163,3.726027,0.657534,24.767123,1.808219,3.013699,6.958904,3.232877
1,2,james-wiseman-1,James Wiseman,memphis,Fr.,1,9,0.769,0.769,1.038,0.704,3,85,69,59,1,0.000,0,32,0.760,7.2,3,0.443,0.579710,5.217391,34.202899,0.579710,0.000000,18.550725,1.739130
2,4,obadiah-toppin-1,Obi Toppin,dayton,So.,111,60,0.674,0.646,0.306,0.713,56,81,1590,926,46,0.398,33,365,0.684,14.6,116,0.244,2.792453,1.509434,23.295597,1.157233,0.830189,9.182390,2.918239
3,5,jaden-mcdaniels-1,Jaden McDaniels,washington,Fr.,49,35,0.453,0.396,0.347,0.741,23,81,733,285,20,0.318,28,135,0.499,21.2,77,0.087,2.673943,1.909959,15.552524,1.091405,1.527967,7.366985,4.201910
4,6,nico-mannion-1,Nico Mannion,arizona,Fr.,126,0,0.477,0.402,0.349,0.813,23,75,716,323,25,0.339,39,61,0.531,16.2,59,0.164,7.039106,0.000000,18.044693,1.396648,2.178771,3.407821,3.296089


In [27]:
# Set Xnew to have the same independent variables as X
Xnew = draftDF[[
                 "field_goal_percentage",
                 "free_throw_attempt_rate",
                 "height", 
                 "three_point_percentage",
                 "true_shooting_percentage",
                 "win_shares_per_40_minutes",
                ]]

Xnew.head()

,field_goal_percentage,free_throw_attempt_rate,height,three_point_percentage,true_shooting_percentage,win_shares_per_40_minutes
0,0.411,0.328,77,0.318,0.526,0.163
1,0.769,1.038,85,0.000,0.760,0.443
2,0.646,0.306,81,0.398,0.684,0.244
3,0.396,0.347,81,0.318,0.499,0.087
4,0.402,0.349,75,0.339,0.531,0.164


In [28]:
# Predict the new Dependent Values

model = load_model("Draft_Machine_NeuralNetwork.h5")
ynew = model.predict_classes(Xnew)
label_encoder.inverse_transform(ynew)

ynew


#counts_elements = np.unique(ynew, return_counts=True)

#print(f"# of Busts(s): {counts_elements[0]} \n# of Players(s): {counts_elements[1]}")

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1], dtype=int64)